In [1]:
import os
import time
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

from torch import autograd
from torch.autograd import Variable
from torch.utils.data import DataLoader

from torchvision import datasets, transforms
from biotorch.initialization.functions import convert_module
from biotorch.layers import Linear, Conv2d

## Define Model

In [2]:
class LeNet(nn.Module):
    """
    Classic LeNet Architecture
    """

    def __init__(self, activation='tanh'):
        """
        :param in_features: dimension of input features (784 for MNIST)
        :param num_layers: number of layers for feed-forward net
        :param num_hidden_list: list of integers indicating hidden nodes of each layer
        """
        super(LeNet, self).__init__()

        if activation == 'relu':
            self.activation = torch.relu
        elif activation == 'tanh':
            self.activation = torch.tanh
        elif activation == 'sigmoid':
            self.activation = torch.sigmoid

        # create layer operations
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(4 * 4 * 50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, inputs):
        """
        forward pass, which is same for conventional feed-forward net
        :param inputs: inputs with shape [batch_size, in_features]
        :return: logit outputs from the network
        """
        inputs = self.activation(self.conv1(inputs))
        inputs = self.pool(inputs)

        inputs = self.activation(self.conv2(inputs))
        inputs = self.pool(inputs)

        inputs = inputs.view(inputs.size()[0], -1)

        inputs = self.activation(self.fc1(inputs))
        inputs = self.fc2(inputs)
        return inputs

In [3]:
class LeNetFA(nn.Module):
    """
    Classic LeNet Architecture
    """

    def __init__(self, activation='tanh'):
        """
        :param in_features: dimension of input features (784 for MNIST)
        :param num_layers: number of layers for feed-forward net
        :param num_hidden_list: list of integers indicating hidden nodes of each layer
        """
        super(LeNetFA, self).__init__()

        if activation == 'relu':
            self.activation = torch.relu
        elif activation == 'tanh':
            self.activation = torch.tanh
        elif activation == 'sigmoid':
            self.activation = torch.sigmoid

        # create layer operations
        self.conv1 = Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1, bias=False)
        self.conv2 = Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = Linear(4 * 4 * 50, 500, bias=False)
        self.fc2 = Linear(500, 10)

    def forward(self, inputs):
        """
        forward pass, which is same for conventional feed-forward net
        :param inputs: inputs with shape [batch_size, in_features]
        :return: logit outputs from the network
        """
        inputs = self.activation(self.conv1(inputs))
        inputs = self.pool(inputs)

        inputs = self.activation(self.conv2(inputs))
        inputs = self.pool(inputs)

        inputs = inputs.view(inputs.size()[0], -1)

        inputs = self.activation(self.fc1(inputs))
        inputs = self.fc2(inputs)
        return inputs

## Test Function

In [4]:
def test(model, test_loader, batch_size, device):
    test_loss = 0
    correct = 0
    # Desactivate the autograd engine in test
    with torch.no_grad():
        for data, target in test_loader:
            #data = data.view(batch_size, -1)
            inputs, targets = data.to(device), target.to(device)
            predictions = model(inputs)
            predictions = torch.squeeze(predictions)
            test_loss += F.nll_loss(predictions, targets, size_average=False).item()
            pred = predictions.data.max(1, keepdim=True)[1]
            correct += pred.eq(targets.data.view_as(pred)).sum()

    test_loss /= len(test_loader.dataset)
    return test_loss, correct

## Training code

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0

batch_size = 32

In [6]:
# set up datasets
print('==> Preparing data..')

train_loader = DataLoader(datasets.MNIST('./data', train=True, download=True,
                                             transform=transforms.Compose([
                                                 transforms.ToTensor(),
                                                 transforms.Normalize((0.1307,), (0.3081,))
                                             ])),
                              batch_size=batch_size, shuffle=True, drop_last=True)

test_loader = DataLoader(datasets.MNIST('./data', train=False, download=True,
                                            transform=transforms.Compose([
                                                transforms.ToTensor(),
                                                transforms.Normalize((0.1307,), (0.3081,))
                                            ])),
                             batch_size=batch_size, shuffle=False, drop_last=True)

==> Preparing data..


In [7]:
cudnn.benchmark = True

## Experiment

In [8]:
# Create Back Propagation Model
model_bp = LeNet()

In [9]:
# Create Feedback Alignment model
model_fa = LeNet()
# model_fa = LeNetFA()
# Uncomment for multiple GPUs
# model_fa = nn.DataParallel(model_fa, device_ids=[0, 1, 2, 3])

In [10]:
convert_module(model_fa)

All the 2 <class 'torch.nn.modules.conv.Conv2d'> layers were converted successfully
All the 2 <class 'torch.nn.modules.linear.Linear'> layers were converted successfully


In [11]:
# you can comment this to run on GPU
device = 'cpu'

In [12]:
for i, layer in enumerate(model_fa.modules()):
    print(type(layer))

<class '__main__.LeNet'>
<class 'biotorch.layers.conv.Conv2d'>
<class 'biotorch.layers.conv.Conv2d'>
<class 'torch.nn.modules.pooling.MaxPool2d'>
<class 'biotorch.layers.linear.Linear'>
<class 'biotorch.layers.linear.Linear'>


In [13]:
device

'cpu'

In [14]:
model_fa.to(device)
model_bp.to(device)

LeNet(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [15]:
# Create optimizers
loss_crossentropy = torch.nn.CrossEntropyLoss()
optimizer_fa = torch.optim.RMSprop(model_fa.parameters(), lr=1e-4, weight_decay=0.)
optimizer_bp = torch.optim.RMSprop(model_bp.parameters(), lr=1e-4, weight_decay=0.)

In [16]:
logger_train = open('results' + 'bp_vs_fa.txt', 'w')

In [17]:
epochs = 10
for epoch in range(epochs):
    for idx_batch, (inputs, targets) in enumerate(train_loader):
        # flatten the inputs from square image to 1d vector
        #inputs = inputs.view(batch_size, -1)
        # wrap them into varaibles
        inputs, targets = inputs.to(device), targets.to(device)
        # get outputs from the model
        #print("inputs = ", inputs.size())
        outputs_fa = model_fa(inputs)
        outputs_bp = model_bp(inputs)
        # print(outputs_fa.size())
        # print(outputs_bp.size())
        # calculate loss
        outputs_fa = torch.squeeze(outputs_fa)
        outputs_bp = torch.squeeze(outputs_bp)
        # print(outputs_fa.size())
        # print(outputs_bp.size())

        # print("-"*20)
        #print("targets.size() = ", targets.size())
        # input()
        
        loss_bp = loss_crossentropy(outputs_bp, targets)
        loss_fa = loss_crossentropy(outputs_fa, targets)
        # print(loss_bp, loss_fa)
        
        t_bp = time.time()
        model_bp.zero_grad()
        loss_bp.backward()
        optimizer_bp.step()
        t_avg_bp = time.time() - t_bp
        
        t_fa = time.time()
        model_fa.zero_grad()
        loss_fa.backward()
        optimizer_fa.step()
        t_avg_fa = time.time() - t_fa

        if (idx_batch + 1) % 100 == 0:
            train_log = 'epoch ' + str(epoch) + ' step ' + str(idx_batch + 1) + \
                        ' loss_fa ' + str(loss_fa.data.item()) + ' loss_bp ' + str(loss_bp.data.item())
                         
            times = ' time_fa '+ str(t_avg_fa) + ' time_bp ' + str(t_avg_bp)
            time_dif = t_avg_fa - t_avg_bp
            print(train_log)
            print(times)
            print(time_dif)
            logger_train.write(train_log + '\n')

    # Test models
    test_loss_fa, correct_fa = test(model_fa, test_loader, batch_size, device)    
    test_loss_bp, correct_bp = test(model_bp, test_loader, batch_size, device)

    print('\n[Epoch {}] Test results'.format(epoch))
    print('\tFA: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(test_loss_fa,
                                                                      correct_fa, len(test_loader.dataset), 100. * correct_fa / len(test_loader.dataset)))
    print('\tBP: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss_bp,
                                                                        correct_bp, len(test_loader.dataset), 100. * correct_bp / len(test_loader.dataset)))

epoch 0 step 100 loss_fa 2.2755868434906006 loss_bp 0.79414963722229
 time_fa 0.005955219268798828 time_bp 0.003373861312866211
0.002581357955932617
epoch 0 step 200 loss_fa 2.055450201034546 loss_bp 0.36971384286880493
 time_fa 0.007066965103149414 time_bp 0.0032753944396972656
0.0037915706634521484
epoch 0 step 300 loss_fa 1.695908546447754 loss_bp 0.2870963513851166
 time_fa 0.00597381591796875 time_bp 0.0043735504150390625
0.0016002655029296875
epoch 0 step 400 loss_fa 1.3032976388931274 loss_bp 0.3816996216773987
 time_fa 0.006037712097167969 time_bp 0.003428936004638672
0.002608776092529297
epoch 0 step 500 loss_fa 0.8894603252410889 loss_bp 0.14889290928840637
 time_fa 0.0057566165924072266 time_bp 0.0032765865325927734
0.002480030059814453
epoch 0 step 600 loss_fa 0.7302398085594177 loss_bp 0.1579393446445465
 time_fa 0.006291627883911133 time_bp 0.010212421417236328
-0.003920793533325195
epoch 0 step 700 loss_fa 1.055591106414795 loss_bp 0.29359927773475647
 time_fa 0.00544667

/home/albert/projects/biotorch/.venv/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



[Epoch 0] Test results
	FA: Avg. loss: -7.2105, Accuracy: 9186/10000 (92%)
	BP: Avg. loss: -8.7185, Accuracy: 9753/10000 (98%)

epoch 1 step 100 loss_fa 0.27238550782203674 loss_bp 0.10038815438747406
 time_fa 0.007376194000244141 time_bp 0.0031604766845703125
0.004215717315673828
epoch 1 step 200 loss_fa 0.2137887179851532 loss_bp 0.04911723732948303
 time_fa 0.0055751800537109375 time_bp 0.003305673599243164
0.0022695064544677734
epoch 1 step 300 loss_fa 0.2731361985206604 loss_bp 0.043234437704086304
 time_fa 0.005491495132446289 time_bp 0.0033669471740722656
0.0021245479583740234
epoch 1 step 400 loss_fa 0.2416723519563675 loss_bp 0.08072543889284134
 time_fa 0.005636692047119141 time_bp 0.003507375717163086
0.0021293163299560547
epoch 1 step 500 loss_fa 0.21130669116973877 loss_bp 0.03591394051909447
 time_fa 0.0060503482818603516 time_bp 0.0033407211303710938
0.002709627151489258
epoch 1 step 600 loss_fa 0.4170858561992645 loss_bp 0.05042315274477005
 time_fa 0.00597429275512695

KeyboardInterrupt: 